In [1]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD, evaluate
import time
import datetime
import random
import numpy as np
import six
from tabulate import tabulate
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise import NormalPredictor
from surprise import BaselineOnly
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNBaseline
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering

%matplotlib inline

plt.rcParams['figure.figsize'] = (6, 4)
plt.style.use('ggplot')
%config InlineBackend.figure_formats = {'png', 'retina'}

/Users/Smile/anaconda/lib/python3.6/site-packages/scipy/__init__.py:115: UserWarning: Numpy 1.13.3 or above is required for this version of scipy (detected version 1.12.1)
  UserWarning)


RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

ImportError: numpy.core.multiarray failed to import

In [ ]:
anime = pd.read_csv('../anime.csv')

In [ ]:
anime.head()

In [ ]:
print(anime.shape)

In [ ]:
user = pd.read_csv('../rating.csv')

In [ ]:
user.head(10)

In [ ]:
print(user.shape)

In [ ]:
#drop the data that rating is -1
user_filtered = user[user.rating!=(-1)]
user_filtered.head()

In [ ]:
#drop the user that user_frequency < 130
user_filtered = user_filtered.groupby('user_id').filter(lambda x : (x['user_id'].count()>=130).any())
user_filtered.head()

In [ ]:
user_filtered.shape

In [ ]:
anime.shape

In [ ]:
#create df of anime < 50000 members
title_ge50000 = anime[anime.members>=50000]
title_ge50000.shape

In [ ]:
title_ge50000.head()

In [ ]:
title_ge50000.anime_id

In [ ]:
user_filtered['user_id']

In [ ]:
#drop any anime < 50000 members
user_filtered = userSVD[userSVD['anime_id'].isin(title_ge50000.anime_id)] 
user_filtered.head()

#drop any user's rating on a droped anime

In [ ]:
user_filtered.shape

In [ ]:
#drop the data that index >10000
user_filtered = user_filtered.reset_index()
user_filtered = user_filtered.drop(columns=['index'])
user_filtered.head()
user_filtered.to_csv('user_filtered.csv',index=False)

In [ ]:
user_filtered.shape

In [ ]:
df_title = anime.drop(['genre','type','episodes','rating','members'], axis=1)
df_title.set_index('anime_id', inplace = True)
print (df_title.head(10))

In [ ]:
#adjust the scale for the function in Surprise
reader = Reader(rating_scale=(0, 10))

# Format the dataset to correspond to required format 
# colume: user id, item id and ratings.
data = Dataset.load_from_df(user_filtered[['user_id', 'anime_id', 'rating']], reader)

#this user likes in the past
df_1 = user_filtered[(user_filtered['user_id'] == 1) & (user_filtered['rating'] == 10)]
df_1 = df_1.set_index('anime_id')
df_1 = df_1.join(anime.name)['name']
print(df_1)

In [ ]:
#prediction what this user will like to watch

#drop all anime that this user has watched
user_1 = df_title.copy()
user_1 = user_1.reset_index()
user_1 = user_1[~user_1['anime_id'].isin(df_title)]

# getting full dataset
##data = Dataset.load_from_df(user[['user_id', 'anime_id', 'rating']], reader)

trainset = data.build_full_trainset()
svd = SVD()
svd.train(trainset)

user_1['Estimate_Score'] = user_1['anime_id'].apply(lambda x: svd.predict(1, x).est)

user_1 = user_1.drop('anime_id', axis = 1)

user_1 = user_1.sort_values('Estimate_Score', ascending=False)
print(user_1.head(10))

In [ ]:
# The algorithms to cross-validate
classes = (SVD, NMF, SlopeOne, KNNBasic, KNNWithMeans, KNNBaseline, CoClustering, BaselineOnly, NormalPredictor)

# ugly dict to map algo names and datasets to their markdown links in the table
stable = 'http://surprise.readthedocs.io/en/stable/'
LINK = {'SVD': '[{}]({})'.format('SVD',
                                 stable +
                                 'matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD'),
        'NMF': '[{}]({})'.format('NMF',
                                stable +
                                'matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.NMF'),
        'SlopeOne': '[{}]({})'.format('Slope One',
                                     stable +
                                     'slope_one.html#surprise.prediction_algorithms.slope_one.SlopeOne'),
        'KNNBasic': '[{}]({})'.format('k-NN',
                                     stable +
                                     'knn_inspired.html#surprise.prediction_algorithms.knns.KNNBasic'),
        'KNNWithMeans': '[{}]({})'.format('Centered k-NN',
                                          stable +
                                          'knn_inspired.html#surprise.prediction_algorithms.knns.KNNWithMeans'),
        'KNNBaseline': '[{}]({})'.format('k-NN Baseline',
                                         stable +
                                         'knn_inspired.html#surprise.prediction_algorithms.knns.KNNBaseline'),
        'CoClustering': '[{}]({})'.format('Co-Clustering',
                                          stable +
                                          'co_clustering.html#surprise.prediction_algorithms.co_clustering.CoClustering'),
        'BaselineOnly': '[{}]({})'.format('Baseline',
                                          stable +
                                          'basic_algorithms.html#surprise.prediction_algorithms.baseline_only.BaselineOnly'),
        'NormalPredictor': '[{}]({})'.format('Random',
                                             stable +
                                             'basic_algorithms.html#surprise.prediction_algorithms.random_pred.NormalPredictor'),
        }


# set RNG
np.random.seed(0)
random.seed(0)

kf = KFold(random_state=0)  # folds will be the same for all algorithms.

table = []
for klass in classes:
    start = time.time()
    out = cross_validate(klass(), data, ['rmse', 'mae'], kf)
    cv_time = str(datetime.timedelta(seconds=int(time.time() - start)))
    link = LINK[klass.__name__]
    mean_rmse = '{:.3f}'.format(np.mean(out['test_rmse']))
    mean_mae = '{:.3f}'.format(np.mean(out['test_mae']))

    new_line = [link, mean_rmse, mean_mae, cv_time]
    print(tabulate([new_line], tablefmt="pipe"))  # print current algo perf
    table.append(new_line)

header = ['data',
          'RMSE',
          'MAE',
          'Time'
          ]
print(tabulate(table, header, tablefmt="pipe"))